In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import json
import time
from tqdm.notebook import tqdm 

<h4>Start cells here if {state}_geo file has not been FIPS encoded

In [2]:
fips_check = pd.read_csv("data/state_data/geo/geo_az.csv") #- original geocoded file
fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
fips_check.head() #15796

,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,...,Veteran,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long
0,2008847101,4/10/20,988,PPP,"SFE HOLDINGS, LLC",9366 East Raintree Drive,SCOTTSDALE,AZ,85260-2098,8/17/21,...,Unanswered,NaN,10108219.18,6/11/21,72.0,Accomodation and Food Services,ARIZONA DISTRICT OFFICE,"9366 East Raintree Drive, SCOTTSDALE, AZ",33.620398,-111.878704
1,3057347105,4/11/20,988,PPP,NAVAJO TRIBAL UTILITY AUTHORITY,PO Box 170,FORT DEFIANCE,AZ,86504,NaN,...,Unanswered,Y,NaN,NaN,22.0,Utilities,ARIZONA DISTRICT OFFICE,"PO Box 170, FORT DEFIANCE, AZ",35.744460,-109.076483
2,3470877104,4/11/20,988,PPP,"HOME HEALTH AGENCY - ARIZONA, LLC",2999 N44th Street Suite 100,Phoenix,AZ,85018-7261,9/25/21,...,Unanswered,NaN,10133388.89,8/17/21,62.0,Health Care and Social Assistance,ARIZONA DISTRICT OFFICE,"2999 N44th Street Suite 100, Phoenix, AZ",33.482323,-111.986368
3,3530847305,4/29/20,988,PPP,"ARIZONA COMMUNITY PHYSICIANS, P.C.",5055 E BROADWAY BLVD Ste A100,TUCSON,AZ,85711-3629,NaN,...,Unanswered,NaN,10131666.67,8/26/21,62.0,Health Care and Social Assistance,ARIZONA DISTRICT OFFICE,"5055 E BROADWAY BLVD Ste A100, TUCSON, AZ",32.222108,-110.884676
4,3771367108,4/12/20,988,PPP,"NEXTIVA, INC.",8880 E CHAPPARAL RD SUITE 300,SCOTTSDALE,AZ,85250,8/7/21,...,Unanswered,NaN,10126027.40,7/22/21,51.0,Information,ARIZONA DISTRICT OFFICE,"8880 E CHAPPARAL RD SUITE 300, SCOTTSDALE, AZ",33.509368,-111.889853


In [3]:
columns = ['LoanNumber','full_add', 'Lat', 'Long']
fips_check = fips_check[columns]

In [4]:
fips_check_nan = fips_check[fips_check["Lat"].isnull()] # check to make sure no nulls in coords
fips_check_nan #5

,LoanNumber,full_add,Lat,Long
958,5478438406,"18716 S Gadsden St, Gadsden, AZ",NaN,NaN
1172,9593207001,"3200 Willow Creek Rd, PRESCOTT, AZ",NaN,NaN
3497,5800237310,"1/4 miles N. Antelope Rd N22B, Page, AZ",NaN,NaN
10038,1837868607,"2221 E. Frontage Rd. Suit B101 TO B105, Tubac, AZ",NaN,NaN


In [5]:
fips_check_notna = fips_check[fips_check["Lat"].notna()] # check to make sure no nulls in coords
fips_check_notna.head() #2658

,LoanNumber,full_add,Lat,Long
0,2008847101,"9366 East Raintree Drive, SCOTTSDALE, AZ",33.620398,-111.878704
1,3057347105,"PO Box 170, FORT DEFIANCE, AZ",35.744460,-109.076483
2,3470877104,"2999 N44th Street Suite 100, Phoenix, AZ",33.482323,-111.986368
3,3530847305,"5055 E BROADWAY BLVD Ste A100, TUCSON, AZ",32.222108,-110.884676
4,3771367108,"8880 E CHAPPARAL RD SUITE 300, SCOTTSDALE, AZ",33.509368,-111.889853


<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [68]:
fips_check_scraped = pd.read_csv("data/state_data/geo/geo_fips/AZ_fips_scraped.csv") 
fips_check_scraped.head()

,Unnamed: 0,Unnamed: 0.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long,FIPS_z
0,0,0,2008847101,4/10/20,988,PPP,"SFE HOLDINGS, LLC",9366 East Raintree Drive,SCOTTSDALE,AZ,...,NaN,10108219.18,6/11/21,72.0,Accomodation and Food Services,ARIZONA DISTRICT OFFICE,"9366 East Raintree Drive, SCOTTSDALE, AZ",33.620398,-111.878704,4.013217e+13
1,1,1,3057347105,4/11/20,988,PPP,NAVAJO TRIBAL UTILITY AUTHORITY,PO Box 170,FORT DEFIANCE,AZ,...,Y,NaN,NaN,22.0,Utilities,ARIZONA DISTRICT OFFICE,"PO Box 170, FORT DEFIANCE, AZ",35.744460,-109.076483,4.001944e+13
2,2,2,3470877104,4/11/20,988,PPP,"HOME HEALTH AGENCY - ARIZONA, LLC",2999 N44th Street Suite 100,Phoenix,AZ,...,NaN,10133388.89,8/17/21,62.0,Health Care and Social Assistance,ARIZONA DISTRICT OFFICE,"2999 N44th Street Suite 100, Phoenix, AZ",33.482323,-111.986368,4.013111e+13
3,3,3,3530847305,4/29/20,988,PPP,"ARIZONA COMMUNITY PHYSICIANS, P.C.",5055 E BROADWAY BLVD Ste A100,TUCSON,AZ,...,NaN,10131666.67,8/26/21,62.0,Health Care and Social Assistance,ARIZONA DISTRICT OFFICE,"5055 E BROADWAY BLVD Ste A100, TUCSON, AZ",32.222108,-110.884676,4.019003e+13
4,4,4,3771367108,4/12/20,988,PPP,"NEXTIVA, INC.",8880 E CHAPPARAL RD SUITE 300,SCOTTSDALE,AZ,...,NaN,10126027.40,7/22/21,51.0,Information,ARIZONA DISTRICT OFFICE,"8880 E CHAPPARAL RD SUITE 300, SCOTTSDALE, AZ",33.509368,-111.889853,4.013941e+13


In [69]:
columns = ['LoanNumber','full_add', 'Lat', 'Long', 'FIPS_z']
fips_check_scraped = fips_check_scraped[columns]

In [ ]:
#pd.set_option('display.float_format', '{:.0f}'.format)

In [70]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() #15792

,LoanNumber,full_add,Lat,Long,FIPS_z
0,2008847101,"9366 East Raintree Drive, SCOTTSDALE, AZ",33.620398,-111.878704,4.013217e+13
1,3057347105,"PO Box 170, FORT DEFIANCE, AZ",35.744460,-109.076483,4.001944e+13
2,3470877104,"2999 N44th Street Suite 100, Phoenix, AZ",33.482323,-111.986368,4.013111e+13
3,3530847305,"5055 E BROADWAY BLVD Ste A100, TUCSON, AZ",32.222108,-110.884676,4.019003e+13
4,3771367108,"8880 E CHAPPARAL RD SUITE 300, SCOTTSDALE, AZ",33.509368,-111.889853,4.013941e+13


In [73]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null.head() #8655

,LoanNumber,full_add,Lat,Long,FIPS_z
736,3445788509,"PRIDE BAKERIES LLC, PHOENIX, AZ",38.740566,-90.297851,NaN
4066,4410207007,"P.O. Box 19460, PHOENIX, AZ",40.118071,-75.539316,NaN
4098,1641998609,"1762 Highway 95, Bullhead City, AZ",43.844102,-116.953670,NaN
5487,8620287005,"6240 Tacoma Mall Blvd. Ste 315, QUEEN CREEK, AZ",47.199526,-122.463532,NaN
6154,4214117203,"1762 HIGHWAY 95, Bullhead City, AZ",43.844102,-116.953670,NaN


<h4>FIPS Script

In [74]:
#if geocode file
#lat = fips_check_notna["Lat"].astype(float).astype(str).to_list()
#long = fips_check_notna["Long"].astype(float).astype(str).to_list()

#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [75]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
    
executable_path = {'executable_path': ChromeDriverManager().install()}
driver = Browser('chrome', **executable_path, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('_x_id').clear()
        driver.find_by_id('_y_id').clear()
        driver.find_by_id('_x_id').fill(long[i])
        driver.find_by_id('_y_id').fill(lat[i])
        driver.find_by_id('proceed').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='pl_gov_census_geo_geocoder_domain_GeographyResult')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('04')] #AZ
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                notfound.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/97.0.4692.71/chromedriver_mac64.zip
Driver has been saved in cache [/Users/ciaraspencer/.wdm/drivers/chromedriver/mac64/97.0.4692.71]


coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [76]:
geo_id_list =geo_id 

Length Check

In [77]:
print(len(geo_id_list))

8655


Matching FIPS to DataFrame

In [78]:
#for geocode file
#fips_fill = fips_check_notna.iloc[:3032] #confirm pair to notna DataFrame
#if FIPS available
fips_fill = fips_check_scraped_null.iloc[:8655]

In [79]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

,LoanNumber,full_add,Lat,Long,FIPS_z
736,3445788509,"PRIDE BAKERIES LLC, PHOENIX, AZ",38.740566,-90.297851,[]
4066,4410207007,"P.O. Box 19460, PHOENIX, AZ",40.118071,-75.539316,[]
4098,1641998609,"1762 Highway 95, Bullhead City, AZ",43.844102,-116.953670,[]
5487,8620287005,"6240 Tacoma Mall Blvd. Ste 315, QUEEN CREEK, AZ",47.199526,-122.463532,[]
6154,4214117203,"1762 HIGHWAY 95, Bullhead City, AZ",43.844102,-116.953670,[]
...,...,...,...,...,...
15791,9422327705,"7164 38TH ST, TUCSON, AZ",32.189429,-110.839316,[040190040372003]
15792,9650217306,"1933 E Janice Way, Phoenix, AZ",33.623513,-112.041885,[040131036074002]
15793,9716018610,"1927 N Gilbert Rd Ste 7, Mesa, AZ",33.450479,-111.787078,[040134206031017]
15794,9821317002,"235 E BASELINE RD, TEMPE, AZ",33.377459,-111.935714,[040133199021002]


Remove brackets

In [80]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

,LoanNumber,full_add,Lat,Long,FIPS_z
736,3445788509,"PRIDE BAKERIES LLC, PHOENIX, AZ",38.740566,-90.297851,NaN
4066,4410207007,"P.O. Box 19460, PHOENIX, AZ",40.118071,-75.539316,NaN
4098,1641998609,"1762 Highway 95, Bullhead City, AZ",43.844102,-116.953670,NaN
5487,8620287005,"6240 Tacoma Mall Blvd. Ste 315, QUEEN CREEK, AZ",47.199526,-122.463532,NaN
6154,4214117203,"1762 HIGHWAY 95, Bullhead City, AZ",43.844102,-116.953670,NaN
...,...,...,...,...,...
15791,9422327705,"7164 38TH ST, TUCSON, AZ",32.189429,-110.839316,040190040372003
15792,9650217306,"1933 E Janice Way, Phoenix, AZ",33.623513,-112.041885,040131036074002
15793,9716018610,"1927 N Gilbert Rd Ste 7, Mesa, AZ",33.450479,-111.787078,040134206031017
15794,9821317002,"235 E BASELINE RD, TEMPE, AZ",33.377459,-111.935714,040133199021002


In [81]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check

,LoanNumber,full_add,Lat,Long,FIPS_z
736,3445788509,"PRIDE BAKERIES LLC, PHOENIX, AZ",38.740566,-90.297851,NaN
4066,4410207007,"P.O. Box 19460, PHOENIX, AZ",40.118071,-75.539316,NaN
4098,1641998609,"1762 Highway 95, Bullhead City, AZ",43.844102,-116.953670,NaN
5487,8620287005,"6240 Tacoma Mall Blvd. Ste 315, QUEEN CREEK, AZ",47.199526,-122.463532,NaN
6154,4214117203,"1762 HIGHWAY 95, Bullhead City, AZ",43.844102,-116.953670,NaN
15103,9476237708,"MILE POST 1215 US 95 N, COURT SIDE, AZ",34.228172,-114.635898,NaN


Parse for Merge

In [82]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
736,3445788509,NaN
4066,4410207007,NaN
4098,1641998609,NaN
5487,8620287005,NaN
6154,4214117203,NaN
...,...,...
15791,9422327705,040190040372003
15792,9650217306,040131036074002
15793,9716018610,040134206031017
15794,9821317002,040133199021002


Read in base data

In [83]:
#if from geocode file
'''
fips_check = pd.read_csv("data/state_data/geo/geo_az.csv")
fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
fips_check.head() #2663
'''

#if from FIPS file
fips_check = pd.read_csv("data/state_data/geo/geo_fips/AZ_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
fips_check.head() #15796

,Unnamed: 0,Unnamed: 0.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long,FIPS_z
0,0,0,2008847101,4/10/20,988,PPP,"SFE HOLDINGS, LLC",9366 East Raintree Drive,SCOTTSDALE,AZ,...,NaN,10108219.18,6/11/21,72.0,Accomodation and Food Services,ARIZONA DISTRICT OFFICE,"9366 East Raintree Drive, SCOTTSDALE, AZ",33.620398,-111.878704,4.013217e+13
1,1,1,3057347105,4/11/20,988,PPP,NAVAJO TRIBAL UTILITY AUTHORITY,PO Box 170,FORT DEFIANCE,AZ,...,Y,NaN,NaN,22.0,Utilities,ARIZONA DISTRICT OFFICE,"PO Box 170, FORT DEFIANCE, AZ",35.744460,-109.076483,4.001944e+13
2,2,2,3470877104,4/11/20,988,PPP,"HOME HEALTH AGENCY - ARIZONA, LLC",2999 N44th Street Suite 100,Phoenix,AZ,...,NaN,10133388.89,8/17/21,62.0,Health Care and Social Assistance,ARIZONA DISTRICT OFFICE,"2999 N44th Street Suite 100, Phoenix, AZ",33.482323,-111.986368,4.013111e+13
3,3,3,3530847305,4/29/20,988,PPP,"ARIZONA COMMUNITY PHYSICIANS, P.C.",5055 E BROADWAY BLVD Ste A100,TUCSON,AZ,...,NaN,10131666.67,8/26/21,62.0,Health Care and Social Assistance,ARIZONA DISTRICT OFFICE,"5055 E BROADWAY BLVD Ste A100, TUCSON, AZ",32.222108,-110.884676,4.019003e+13
4,4,4,3771367108,4/12/20,988,PPP,"NEXTIVA, INC.",8880 E CHAPPARAL RD SUITE 300,SCOTTSDALE,AZ,...,NaN,10126027.40,7/22/21,51.0,Information,ARIZONA DISTRICT OFFICE,"8880 E CHAPPARAL RD SUITE 300, SCOTTSDALE, AZ",33.509368,-111.889853,4.013941e+13


Merge with MainFrame

In [84]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,Unnamed: 0.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,0,0,2008847101,4/10/20,988,PPP,"SFE HOLDINGS, LLC",9366 East Raintree Drive,SCOTTSDALE,AZ,...,10108219.18,6/11/21,72.0,Accomodation and Food Services,ARIZONA DISTRICT OFFICE,"9366 East Raintree Drive, SCOTTSDALE, AZ",33.620398,-111.878704,4.013217e+13,NaN
1,1,1,3057347105,4/11/20,988,PPP,NAVAJO TRIBAL UTILITY AUTHORITY,PO Box 170,FORT DEFIANCE,AZ,...,NaN,NaN,22.0,Utilities,ARIZONA DISTRICT OFFICE,"PO Box 170, FORT DEFIANCE, AZ",35.744460,-109.076483,4.001944e+13,NaN
2,2,2,3470877104,4/11/20,988,PPP,"HOME HEALTH AGENCY - ARIZONA, LLC",2999 N44th Street Suite 100,Phoenix,AZ,...,10133388.89,8/17/21,62.0,Health Care and Social Assistance,ARIZONA DISTRICT OFFICE,"2999 N44th Street Suite 100, Phoenix, AZ",33.482323,-111.986368,4.013111e+13,NaN
3,3,3,3530847305,4/29/20,988,PPP,"ARIZONA COMMUNITY PHYSICIANS, P.C.",5055 E BROADWAY BLVD Ste A100,TUCSON,AZ,...,10131666.67,8/26/21,62.0,Health Care and Social Assistance,ARIZONA DISTRICT OFFICE,"5055 E BROADWAY BLVD Ste A100, TUCSON, AZ",32.222108,-110.884676,4.019003e+13,NaN
4,4,4,3771367108,4/12/20,988,PPP,"NEXTIVA, INC.",8880 E CHAPPARAL RD SUITE 300,SCOTTSDALE,AZ,...,10126027.40,7/22/21,51.0,Information,ARIZONA DISTRICT OFFICE,"8880 E CHAPPARAL RD SUITE 300, SCOTTSDALE, AZ",33.509368,-111.889853,4.013941e+13,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15791,15791,15791,9422327705,5/1/20,988,PPP,"APPLIED RITE DOORS &AMP; DOCKS, INC.",7164 38TH ST,TUCSON,AZ,...,152038.36,9/15/21,23.0,Construction,ARIZONA DISTRICT OFFICE,"7164 38TH ST, TUCSON, AZ",32.189429,-110.839316,NaN,040190040372003
15792,15792,15792,9650217306,5/2/20,988,PPP,TAIBEEYAH MANAGEMENT LLC,1933 E Janice Way,Phoenix,AZ,...,151350.00,4/16/21,54.0,"Professional, Scientific, and Technical Services",ARIZONA DISTRICT OFFICE,"1933 E Janice Way, Phoenix, AZ",33.623513,-112.041885,NaN,040131036074002
15793,15793,15793,9716018610,3/26/21,988,PPS,KTR MESA ENTERPRISES LLC,1927 N Gilbert Rd Ste 7,Mesa,AZ,...,150947.33,11/17/21,71.0,"Arts, Entertainment, and Recreation",ARIZONA DISTRICT OFFICE,"1927 N Gilbert Rd Ste 7, Mesa, AZ",33.450479,-111.787078,NaN,040134206031017
15794,15794,15794,9821317002,4/9/20,988,PPP,BDIT HOLDINGS LLC,235 E BASELINE RD,TEMPE,AZ,...,150812.50,11/3/20,44.0,Retail Trade,ARIZONA DISTRICT OFFICE,"235 E BASELINE RD, TEMPE, AZ",33.377459,-111.935714,NaN,040133199021002


Fill FIPS Columns

In [85]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge

,Unnamed: 0,Unnamed: 0.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,0,0,2008847101,4/10/20,988,PPP,"SFE HOLDINGS, LLC",9366 East Raintree Drive,SCOTTSDALE,AZ,...,6/11/21,72.0,Accomodation and Food Services,ARIZONA DISTRICT OFFICE,"9366 East Raintree Drive, SCOTTSDALE, AZ",33.620398,-111.878704,4.013217e+13,NaN,4.01322e+13
1,1,1,3057347105,4/11/20,988,PPP,NAVAJO TRIBAL UTILITY AUTHORITY,PO Box 170,FORT DEFIANCE,AZ,...,NaN,22.0,Utilities,ARIZONA DISTRICT OFFICE,"PO Box 170, FORT DEFIANCE, AZ",35.744460,-109.076483,4.001944e+13,NaN,4.00194e+13
2,2,2,3470877104,4/11/20,988,PPP,"HOME HEALTH AGENCY - ARIZONA, LLC",2999 N44th Street Suite 100,Phoenix,AZ,...,8/17/21,62.0,Health Care and Social Assistance,ARIZONA DISTRICT OFFICE,"2999 N44th Street Suite 100, Phoenix, AZ",33.482323,-111.986368,4.013111e+13,NaN,4.01311e+13
3,3,3,3530847305,4/29/20,988,PPP,"ARIZONA COMMUNITY PHYSICIANS, P.C.",5055 E BROADWAY BLVD Ste A100,TUCSON,AZ,...,8/26/21,62.0,Health Care and Social Assistance,ARIZONA DISTRICT OFFICE,"5055 E BROADWAY BLVD Ste A100, TUCSON, AZ",32.222108,-110.884676,4.019003e+13,NaN,4.019e+13
4,4,4,3771367108,4/12/20,988,PPP,"NEXTIVA, INC.",8880 E CHAPPARAL RD SUITE 300,SCOTTSDALE,AZ,...,7/22/21,51.0,Information,ARIZONA DISTRICT OFFICE,"8880 E CHAPPARAL RD SUITE 300, SCOTTSDALE, AZ",33.509368,-111.889853,4.013941e+13,NaN,4.01394e+13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15791,15791,15791,9422327705,5/1/20,988,PPP,"APPLIED RITE DOORS &AMP; DOCKS, INC.",7164 38TH ST,TUCSON,AZ,...,9/15/21,23.0,Construction,ARIZONA DISTRICT OFFICE,"7164 38TH ST, TUCSON, AZ",32.189429,-110.839316,NaN,040190040372003,040190040372003
15792,15792,15792,9650217306,5/2/20,988,PPP,TAIBEEYAH MANAGEMENT LLC,1933 E Janice Way,Phoenix,AZ,...,4/16/21,54.0,"Professional, Scientific, and Technical Services",ARIZONA DISTRICT OFFICE,"1933 E Janice Way, Phoenix, AZ",33.623513,-112.041885,NaN,040131036074002,040131036074002
15793,15793,15793,9716018610,3/26/21,988,PPS,KTR MESA ENTERPRISES LLC,1927 N Gilbert Rd Ste 7,Mesa,AZ,...,11/17/21,71.0,"Arts, Entertainment, and Recreation",ARIZONA DISTRICT OFFICE,"1927 N Gilbert Rd Ste 7, Mesa, AZ",33.450479,-111.787078,NaN,040134206031017,040134206031017
15794,15794,15794,9821317002,4/9/20,988,PPP,BDIT HOLDINGS LLC,235 E BASELINE RD,TEMPE,AZ,...,11/3/20,44.0,Retail Trade,ARIZONA DISTRICT OFFICE,"235 E BASELINE RD, TEMPE, AZ",33.377459,-111.935714,NaN,040133199021002,040133199021002


Drop Extra FIPS Columns

In [86]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop

,Unnamed: 0,Unnamed: 0.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long,FIPS_z
0,0,0,2008847101,4/10/20,988,PPP,"SFE HOLDINGS, LLC",9366 East Raintree Drive,SCOTTSDALE,AZ,...,NaN,10108219.18,6/11/21,72.0,Accomodation and Food Services,ARIZONA DISTRICT OFFICE,"9366 East Raintree Drive, SCOTTSDALE, AZ",33.620398,-111.878704,4.01322e+13
1,1,1,3057347105,4/11/20,988,PPP,NAVAJO TRIBAL UTILITY AUTHORITY,PO Box 170,FORT DEFIANCE,AZ,...,Y,NaN,NaN,22.0,Utilities,ARIZONA DISTRICT OFFICE,"PO Box 170, FORT DEFIANCE, AZ",35.744460,-109.076483,4.00194e+13
2,2,2,3470877104,4/11/20,988,PPP,"HOME HEALTH AGENCY - ARIZONA, LLC",2999 N44th Street Suite 100,Phoenix,AZ,...,NaN,10133388.89,8/17/21,62.0,Health Care and Social Assistance,ARIZONA DISTRICT OFFICE,"2999 N44th Street Suite 100, Phoenix, AZ",33.482323,-111.986368,4.01311e+13
3,3,3,3530847305,4/29/20,988,PPP,"ARIZONA COMMUNITY PHYSICIANS, P.C.",5055 E BROADWAY BLVD Ste A100,TUCSON,AZ,...,NaN,10131666.67,8/26/21,62.0,Health Care and Social Assistance,ARIZONA DISTRICT OFFICE,"5055 E BROADWAY BLVD Ste A100, TUCSON, AZ",32.222108,-110.884676,4.019e+13
4,4,4,3771367108,4/12/20,988,PPP,"NEXTIVA, INC.",8880 E CHAPPARAL RD SUITE 300,SCOTTSDALE,AZ,...,NaN,10126027.40,7/22/21,51.0,Information,ARIZONA DISTRICT OFFICE,"8880 E CHAPPARAL RD SUITE 300, SCOTTSDALE, AZ",33.509368,-111.889853,4.01394e+13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15791,15791,15791,9422327705,5/1/20,988,PPP,"APPLIED RITE DOORS &AMP; DOCKS, INC.",7164 38TH ST,TUCSON,AZ,...,NaN,152038.36,9/15/21,23.0,Construction,ARIZONA DISTRICT OFFICE,"7164 38TH ST, TUCSON, AZ",32.189429,-110.839316,040190040372003
15792,15792,15792,9650217306,5/2/20,988,PPP,TAIBEEYAH MANAGEMENT LLC,1933 E Janice Way,Phoenix,AZ,...,NaN,151350.00,4/16/21,54.0,"Professional, Scientific, and Technical Services",ARIZONA DISTRICT OFFICE,"1933 E Janice Way, Phoenix, AZ",33.623513,-112.041885,040131036074002
15793,15793,15793,9716018610,3/26/21,988,PPS,KTR MESA ENTERPRISES LLC,1927 N Gilbert Rd Ste 7,Mesa,AZ,...,NaN,150947.33,11/17/21,71.0,"Arts, Entertainment, and Recreation",ARIZONA DISTRICT OFFICE,"1927 N Gilbert Rd Ste 7, Mesa, AZ",33.450479,-111.787078,040134206031017
15794,15794,15794,9821317002,4/9/20,988,PPP,BDIT HOLDINGS LLC,235 E BASELINE RD,TEMPE,AZ,...,NaN,150812.50,11/3/20,44.0,Retail Trade,ARIZONA DISTRICT OFFICE,"235 E BASELINE RD, TEMPE, AZ",33.377459,-111.935714,040133199021002


In [87]:
fips_merge_drop.loc[fips_merge_drop["FIPS_z"].notna()] #verify 

,Unnamed: 0,Unnamed: 0.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long,FIPS_z
0,0,0,2008847101,4/10/20,988,PPP,"SFE HOLDINGS, LLC",9366 East Raintree Drive,SCOTTSDALE,AZ,...,NaN,10108219.18,6/11/21,72.0,Accomodation and Food Services,ARIZONA DISTRICT OFFICE,"9366 East Raintree Drive, SCOTTSDALE, AZ",33.620398,-111.878704,4.01322e+13
1,1,1,3057347105,4/11/20,988,PPP,NAVAJO TRIBAL UTILITY AUTHORITY,PO Box 170,FORT DEFIANCE,AZ,...,Y,NaN,NaN,22.0,Utilities,ARIZONA DISTRICT OFFICE,"PO Box 170, FORT DEFIANCE, AZ",35.744460,-109.076483,4.00194e+13
2,2,2,3470877104,4/11/20,988,PPP,"HOME HEALTH AGENCY - ARIZONA, LLC",2999 N44th Street Suite 100,Phoenix,AZ,...,NaN,10133388.89,8/17/21,62.0,Health Care and Social Assistance,ARIZONA DISTRICT OFFICE,"2999 N44th Street Suite 100, Phoenix, AZ",33.482323,-111.986368,4.01311e+13
3,3,3,3530847305,4/29/20,988,PPP,"ARIZONA COMMUNITY PHYSICIANS, P.C.",5055 E BROADWAY BLVD Ste A100,TUCSON,AZ,...,NaN,10131666.67,8/26/21,62.0,Health Care and Social Assistance,ARIZONA DISTRICT OFFICE,"5055 E BROADWAY BLVD Ste A100, TUCSON, AZ",32.222108,-110.884676,4.019e+13
4,4,4,3771367108,4/12/20,988,PPP,"NEXTIVA, INC.",8880 E CHAPPARAL RD SUITE 300,SCOTTSDALE,AZ,...,NaN,10126027.40,7/22/21,51.0,Information,ARIZONA DISTRICT OFFICE,"8880 E CHAPPARAL RD SUITE 300, SCOTTSDALE, AZ",33.509368,-111.889853,4.01394e+13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15791,15791,15791,9422327705,5/1/20,988,PPP,"APPLIED RITE DOORS &AMP; DOCKS, INC.",7164 38TH ST,TUCSON,AZ,...,NaN,152038.36,9/15/21,23.0,Construction,ARIZONA DISTRICT OFFICE,"7164 38TH ST, TUCSON, AZ",32.189429,-110.839316,040190040372003
15792,15792,15792,9650217306,5/2/20,988,PPP,TAIBEEYAH MANAGEMENT LLC,1933 E Janice Way,Phoenix,AZ,...,NaN,151350.00,4/16/21,54.0,"Professional, Scientific, and Technical Services",ARIZONA DISTRICT OFFICE,"1933 E Janice Way, Phoenix, AZ",33.623513,-112.041885,040131036074002
15793,15793,15793,9716018610,3/26/21,988,PPS,KTR MESA ENTERPRISES LLC,1927 N Gilbert Rd Ste 7,Mesa,AZ,...,NaN,150947.33,11/17/21,71.0,"Arts, Entertainment, and Recreation",ARIZONA DISTRICT OFFICE,"1927 N Gilbert Rd Ste 7, Mesa, AZ",33.450479,-111.787078,040134206031017
15794,15794,15794,9821317002,4/9/20,988,PPP,BDIT HOLDINGS LLC,235 E BASELINE RD,TEMPE,AZ,...,NaN,150812.50,11/3/20,44.0,Retail Trade,ARIZONA DISTRICT OFFICE,"235 E BASELINE RD, TEMPE, AZ",33.377459,-111.935714,040133199021002


In [88]:
fips_AZ = fips_merge_drop

Convert to File

In [4]:
fips_AZ.to_csv("data/state_data/geo/geo_fips/AZ_fips_scraped.csv") 

Review & Compare

In [2]:
geo_AZ_fips = pd.read_csv("data/state_data/geo/geo_fips/AZ_fips_scraped.csv") 
geo_AZ_fips 

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,...,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long,FIPS_z
0,0,0,0,0,0,2008847101,4/10/20,988,PPP,"SFE HOLDINGS, LLC",...,N,10108219.18,6/11/21,72.0,Accomodation and Food Services,ARIZONA DISTRICT OFFICE,"9366 East Raintree Drive, SCOTTSDALE, AZ",33.620397,-111.878704,4.013217e+13
1,1,1,1,1,1,3057347105,4/11/20,988,PPP,NAVAJO TRIBAL UTILITY AUTHORITY,...,Y,NaN,NaN,22.0,Utilities,ARIZONA DISTRICT OFFICE,"PO Box 170, FORT DEFIANCE, AZ",35.744460,-109.076483,4.001944e+13
2,2,2,2,2,2,3470877104,4/11/20,988,PPP,"HOME HEALTH AGENCY - ARIZONA, LLC",...,N,10133388.89,8/17/21,62.0,Health Care and Social Assistance,ARIZONA DISTRICT OFFICE,"2999 N44th Street Suite 100, Phoenix, AZ",33.482323,-111.986368,4.013111e+13
3,3,3,3,3,3,3530847305,4/29/20,988,PPP,"ARIZONA COMMUNITY PHYSICIANS, P.C.",...,N,10131666.67,8/26/21,62.0,Health Care and Social Assistance,ARIZONA DISTRICT OFFICE,"5055 E BROADWAY BLVD Ste A100, TUCSON, AZ",32.222108,-110.884676,4.019003e+13
4,4,4,4,4,4,3771367108,4/12/20,988,PPP,"NEXTIVA, INC.",...,N,10126027.40,7/22/21,51.0,Information,ARIZONA DISTRICT OFFICE,"8880 E CHAPPARAL RD SUITE 300, SCOTTSDALE, AZ",33.509368,-111.889852,4.013941e+13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15791,15791,15791,15791,15791,15791,9422327705,5/1/20,988,PPP,"APPLIED RITE DOORS &AMP; DOCKS, INC.",...,N,152038.36,9/15/21,23.0,Construction,ARIZONA DISTRICT OFFICE,"7164 38TH ST, TUCSON, AZ",32.189429,-110.839316,4.019004e+13
15792,15792,15792,15792,15792,15792,9650217306,5/2/20,988,PPP,TAIBEEYAH MANAGEMENT LLC,...,N,151350.00,4/16/21,54.0,"Professional, Scientific, and Technical Services",ARIZONA DISTRICT OFFICE,"1933 E Janice Way, Phoenix, AZ",33.623513,-112.041885,4.013104e+13
15793,15793,15793,15793,15793,15793,9716018610,3/26/21,988,PPS,KTR MESA ENTERPRISES LLC,...,N,150947.33,11/17/21,71.0,"Arts, Entertainment, and Recreation",ARIZONA DISTRICT OFFICE,"1927 N Gilbert Rd Ste 7, Mesa, AZ",33.450479,-111.787078,4.013421e+13
15794,15794,15794,15794,15794,15794,9821317002,4/9/20,988,PPP,BDIT HOLDINGS LLC,...,N,150812.50,11/3/20,44.0,Retail Trade,ARIZONA DISTRICT OFFICE,"235 E BASELINE RD, TEMPE, AZ",33.377459,-111.935714,4.013320e+13


In [3]:
#fill in missing Lat, Long, FIPS
geo_AZ_fips.loc[geo_AZ_fips["FIPS_z"].isnull()]

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,...,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long,FIPS_z
736,736,736,736,736,736,3445788509,2/24/21,988,PPS,PRIDE BAKERIES LLC,...,N,NaN,NaN,72.0,Accomodation and Food Services,ARIZONA DISTRICT OFFICE,"PRIDE BAKERIES LLC, PHOENIX, AZ",38.740566,-90.297852,NaN
958,958,958,958,958,958,5478438406,2/8/21,988,PPS,ORTIZ HARVESTING INC.,...,N,NaN,NaN,11.0,"Agriculture, Forestry, Fishing and Hunting",ARIZONA DISTRICT OFFICE,"18716 S Gadsden St, Gadsden, AZ",NaN,NaN,NaN
1172,1172,1172,1172,1172,1172,9593207001,4/9/20,988,PPP,"FINDLAY TY PRESCOTT, LLC",...,N,1285808.20,6/11/21,44.0,Retail Trade,ARIZONA DISTRICT OFFICE,"3200 Willow Creek Rd, PRESCOTT, AZ",NaN,NaN,NaN
3497,3497,3497,3497,3497,3497,5800237310,4/30/20,988,PPP,KENS TOURS LLC,...,N,584473.43,5/12/21,56.0,Administrative and Support and Waste Managemen...,ARIZONA DISTRICT OFFICE,"1/4 miles N. Antelope Rd N22B, Page, AZ",NaN,NaN,NaN
4066,4066,4066,4066,4066,4066,4410207007,4/3/20,988,PPP,"BACK TO NATURE LANDCARE, INC",...,N,515272.53,2/9/21,56.0,Administrative and Support and Waste Managemen...,ARIZONA DISTRICT OFFICE,"P.O. Box 19460, PHOENIX, AZ",40.118071,-75.539316,NaN
4098,4098,4098,4098,4098,4098,1641998609,3/13/21,988,PPS,INDOMEX DINING LLC,...,N,510516.83,10/19/21,72.0,Accomodation and Food Services,ARIZONA DISTRICT OFFICE,"1762 Highway 95, Bullhead City, AZ",43.844102,-116.953670,NaN
5487,5487,5487,5487,5487,5487,8620287005,4/8/20,988,PPP,"EMERALD CITY MEDICAL STAFFING, INC.",...,N,403079.45,1/20/21,56.0,Administrative and Support and Waste Managemen...,ARIZONA DISTRICT OFFICE,"6240 Tacoma Mall Blvd. Ste 315, QUEEN CREEK, AZ",47.199526,-122.463532,NaN
6154,6154,6154,6154,6154,6154,4214117203,4/27/20,988,PPP,INDOMEX DINING LLC,...,N,365578.18,3/17/21,72.0,Accomodation and Food Services,ARIZONA DISTRICT OFFICE,"1762 HIGHWAY 95, Bullhead City, AZ",43.844102,-116.953670,NaN
10038,10038,10038,10038,10038,10038,1837868607,3/13/21,988,PPS,RUBEN MONROY ARAUX,...,N,NaN,NaN,72.0,Accomodation and Food Services,ARIZONA DISTRICT OFFICE,"2221 E. Frontage Rd. Suit B101 TO B105, Tubac, AZ",NaN,NaN,NaN
15103,15103,15103,15103,15103,15103,9476237708,5/1/20,988,PPP,"BONANZA EXPLORATIONS, INC.",...,N,158189.95,8/16/21,72.0,Accomodation and Food Services,ARIZONA DISTRICT OFFICE,"MILE POST 1215 US 95 N, COURT SIDE, AZ",34.228172,-114.635898,NaN
